# ReadMe
## Summary
- notebook for some figures

## Prerequisite
- Run repo's `notebook/vilio_gradient.ipynb` to obtain vilio results and intermediate files
- place LLama-2 result as `DRIVE_DIR/LLM_PATH`


# init

## env

In [ ]:
!pip install optuna==3.3.0

## modules and variables

In [ ]:
from copy import deepcopy
import glob
import os
from typing import List
import warnings

import dask.dataframe as dd
from dask import delayed
# import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import optuna.integration.lightgbm as opt_lgb
import pandas as pd
import plotly.express as px
# from sklearn.decomposition import PCA
# from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

%matplotlib inline
warnings.simplefilter("ignore")

In [ ]:
# /content/drive/MyDrive/vilio/export/V/attattr
DRIVE_DIR="/content/drive/MyDrive"
LLM_PATH="hf/meta_result.csv"
VILIO_DIR="vilio/export"
VILIO_MODELS=["O","U","V"]
SCORE_TYPE="attattr"
SUBMODELS={
    "O": ["O36","O50","OV50"],
    "U": ["U36","U50","U72"],
    "V": ["V135","V45","V90"],
}
INPUT_MODALITIES=["img","txt"]
INTERACTION_MODALITIES=["cross","image","text"]
EXPORT_DIR="additional_figures"
SPLIT="dev_seen"
RS=1991

In [ ]:
ATTR_COL="score"
PRED_COL="micace"
plot_cols={ATTR_COL: "MIDAS", PRED_COL: "miATE"}

KEEP_COLS=["id","input_modality","model_type","submodel","micace","score"]
KEY_COLS=["id","input_modality","model_type","submodel"]

dummy_cols = deepcopy(KEY_COLS)
dummy_cols.remove("id")

In [ ]:
os.makedirs(f"{DRIVE_DIR}/{VILIO_DIR}/{EXPORT_DIR}", exist_ok=True)

## functions

In [ ]:
def collect_lgb_results(drive_dir=DRIVE_DIR,vilio_dir=VILIO_DIR,export_dir=EXPORT_DIR,score_type=SCORE_TYPE,search_str=""):
    results = glob.glob(f"{drive_dir}/{vilio_dir}/{export_dir}/*{search_str}*.csv")
    return results

@delayed
def read_csv_w_meta(file_path: str):
    df = pd.read_csv(file_path)
    cols = df.columns.tolist()
    filename_seg = file_path.split("/")[-1].split("_")
    df["model_type"], df["seed"] = filename_seg[0], filename_seg[-1].split(".")[0]
    return df[["model_type","seed"]+cols]

def read_csvs_w_meta(file_list: List[str],):
    ddf = dd.from_delayed(
        [read_csv_w_meta(file_path) for file_path in file_list]
    )
    df = ddf.compute()
    return df

def lgb_results2df(search_strs = ["_indv_optuna_occurrences_","_indv_optuna_nunique_"]):
    dfs = {s: None for s in search_strs}
    for search_str in search_strs:
        file_list = collect_lgb_results(search_str=search_str)
        df = read_csvs_w_meta(file_list)
        dfs[search_str] = df.copy()
    return dfs

def collect_vilio_results(drive_dir=DRIVE_DIR,vilio_dir=VILIO_DIR,score_type=SCORE_TYPE,split=SPLIT):
    results = glob.glob(f"{drive_dir}/{vilio_dir}/*/{score_type}/*{split}_result_*.csv")
    return results

@delayed
def read_csv_w_source(file_path: str):
    df = pd.read_csv(file_path)
    df["model_type"], df["score_type"], file_name = file_path.split("/")[-3:]
    df["input_modality"], df["submodel"] = file_name.split("_")[:2]
    df["interaction_type"] = file_name.split("_")[-1].split(".")[0]
    return df.drop_duplicates()

def read_csvs_w_source(file_list: List[str]):
    ddf = dd.from_delayed(
        [read_csv_w_source(file_path) for file_path in file_list]
    )
    df = ddf.compute()
    return df

def vilio_results2df():
    file_list = collect_vilio_results()
    df = read_csvs_w_source(file_list)
    return df

def ate_score(y_true: np.array, y_pred: np.array):
    score_pos = y_pred[y_true==1].sum()/len(y_pred[y_true==1])
    score_neg = y_pred[y_true==0].sum()/len(y_pred[y_true==0])
    return score_pos-score_neg

def convert_metric_to_method(metric: str):
    if metric=="ate":
        out = ate_score
    elif metric=="acc":
        out = accuracy_score
    else:
        raise NotImplementedError
    return out

def score_over_segment(df: pd.DataFrame, gt_col: str, pred_col: str, segment_col: str, metric: str="ate"):
    y_true,y_pred,segment = df[gt_col].values,df[pred_col].values,df[segment_col].values
    out, out_col = [], [segment_col, "cnt", metric]
    for s in sorted(np.unique(segment)):
        cnt = len(y_true[segment==s])
        calculate_score = convert_metric_to_method(metric)
        score = calculate_score(y_true[segment==s], y_pred[segment==s])
        out.append([s, cnt, score])
    out_df = pd.DataFrame(out, columns=out_col)
    return out_df

# load data

In [ ]:
# mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# vilio result
df_vilio = vilio_results2df()
df_vilio.head()

In [ ]:
# llm result
df_llm = pd.read_csv(f"{DRIVE_DIR}/{LLM_PATH}")
df_llm.head()

# analysis

## llm

In [ ]:
np.random.seed(seed=RS)
df_llm["random_label"] = np.random.randint(2, size=len(df_llm))

In [ ]:
df_llm["is_hateful_or_sarcastic"] = (np.logical_or(df_llm["is_hateful"], df_llm["is_sarcastic"])).astype(int)
df_llm["is_few_shot"] = 0
df_llm.loc[df_llm["few_shot_num"] != 0, "is_few_shot"] = 1

In [ ]:
df_llm["is_few_shot"] = 0
df_llm.loc[df_llm["few_shot_num"] != 0, "is_few_shot"] = 1

In [ ]:
# label TTC-absent sample as incorrect
df_llm["gt_flipped"] = df_llm["ground_truth"].apply(lambda x: int(not x))
df_llm.loc[df_llm["is_functional"]==0, ["is_functional", "ground_truth", "is_hateful"]].value_counts()

In [ ]:
# use flipped gt label in case of non-functional response
df_llm["is_hateful_ttc"] = df_llm["is_hateful"].copy()
df_llm.loc[df_llm["is_functional"]==0, "is_hateful_ttc"] = df_llm.loc[df_llm["is_functional"]==0, "gt_flipped"]
df_llm.loc[df_llm["is_functional"]==0, ["is_functional", "ground_truth", "is_hateful", "is_hateful_ttc"]].value_counts()


In [ ]:
# use flipped gt label in case of non-functional response
df_llm["is_hateful_or_sarcastic_ttc"] = df_llm["is_hateful_or_sarcastic"].copy()
df_llm.loc[df_llm["is_functional"]==0, "is_hateful_or_sarcastic_ttc"] = df_llm.loc[df_llm["is_functional"]==0, "gt_flipped"]
df_llm.loc[df_llm["is_functional"]==0, ["is_functional", "ground_truth", "is_hateful_or_sarcastic", "is_hateful_or_sarcastic_ttc"]].value_counts()

In [ ]:
df_llm_ttc = df_llm[df_llm["is_functional"]==1].reset_index(drop=True)

In [ ]:
# before flipping
for metric in ["acc","ate"]:
    print(metric)
    print("all result")
    print("random")
    print(score_over_segment(df_llm, "ground_truth", "random_label", "few_shot_num", metric=metric))
    print("model")
    print(score_over_segment(df_llm, "ground_truth", "is_hateful", "few_shot_num", metric=metric))
    print("---------------------------------------")
    print("ttc")
    print("random")
    print(score_over_segment(df_llm_ttc, "ground_truth", "random_label", "few_shot_num", metric=metric))
    print("model")
    print(score_over_segment(df_llm_ttc, "ground_truth", "is_hateful", "few_shot_num", metric=metric))
    print("=======================================")

In [ ]:
# before flipping - few_shot or not
for metric in ["acc","ate"]:
    print(metric)
    print("all result")
    print("random")
    print(score_over_segment(df_llm, "ground_truth", "random_label", "is_few_shot", metric=metric))
    print("model")
    print(score_over_segment(df_llm, "ground_truth", "is_hateful", "is_few_shot", metric=metric))
    print("---------------------------------------")
    print("ttc")
    print("random")
    print(score_over_segment(df_llm_ttc, "ground_truth", "random_label", "is_few_shot", metric=metric))
    print("model")
    print(score_over_segment(df_llm_ttc, "ground_truth", "is_hateful", "is_few_shot", metric=metric))
    print("=======================================")

In [ ]:
# after flipping
for metric in ["acc","ate"]:
    print(metric)
    print("all result")
    print("random")
    print(score_over_segment(df_llm, "ground_truth", "random_label", "few_shot_num", metric=metric))
    print("model")
    print(score_over_segment(df_llm, "ground_truth", "is_hateful_ttc", "few_shot_num", metric=metric))
    print("---------------------------------------")
    print("ttc")
    print("random")
    print(score_over_segment(df_llm_ttc, "ground_truth", "random_label", "few_shot_num", metric=metric))
    print("model")
    print(score_over_segment(df_llm_ttc, "ground_truth", "is_hateful_ttc", "few_shot_num", metric=metric))
    print("=======================================")

In [ ]:
# after flipping - few_shot or not
for metric in ["acc","ate"]:
    print(metric)
    print("all result")
    print("random")
    print(score_over_segment(df_llm, "ground_truth", "random_label", "is_few_shot", metric=metric))
    print("model")
    print(score_over_segment(df_llm, "ground_truth", "is_hateful_ttc", "is_few_shot", metric=metric))
    print("---------------------------------------")
    print("ttc")
    print("random")
    print(score_over_segment(df_llm_ttc, "ground_truth", "random_label", "is_few_shot", metric=metric))
    print("model")
    print(score_over_segment(df_llm_ttc, "ground_truth", "is_hateful_ttc", "is_few_shot", metric=metric))
    print("=======================================")

In [ ]:
# after flipping - hateful+sarcastic
for metric in ["acc","ate"]:
    print(metric)
    print("all result")
    # print("random")
    # print(score_over_segment(df_llm, "ground_truth", "random_label", "few_shot_num", metric=metric))
    print("model")
    print(score_over_segment(df_llm, "ground_truth", "is_hateful_or_sarcastic_ttc", "few_shot_num", metric=metric))
    print("---------------------------------------")
    print("ttc")
    # print("random")
    # print(score_over_segment(df_llm_ttc, "ground_truth", "random_label", "few_shot_num", metric=metric))
    print("model")
    print(score_over_segment(df_llm_ttc, "ground_truth", "is_hateful_or_sarcastic_ttc", "few_shot_num", metric=metric))
    print("=======================================")

## Vilio

### preprocessing

#### shared 1

In [ ]:
# limit scope
score_type="attattr"
# input_modality="txt"
# submodel="U72"
df_scope = df_vilio[
    (df_vilio["score_type"]==score_type)
    # &(df_horizontal["input_modality"]==input_modality)
    # &(df_horizontal["submodel"]==submodel)
].drop("score_type", axis=1).reset_index(drop=True)
df_scope.head()

In [ ]:
# transpose
for i,it in enumerate(df_scope["interaction_type"].unique()):
    df_it = df_scope.loc[df_scope["interaction_type"]==it, KEEP_COLS].reset_index(drop=True).rename(plot_cols, axis=1)
    if not i:
        it_dict = {"MIDAS": f"MIDAS_{it}"}
        df_horizontal = df_it.copy()
    else:
        df_it = df_it.set_index(KEY_COLS).drop("miATE", axis=1)
        df_horizontal = df_horizontal.merge(df_it, left_on=KEY_COLS, right_index=True, suffixes=["", f"_{it}"])
df_horizontal = df_horizontal.rename(it_dict, axis=1)
df_horizontal.head()

In [ ]:
# target variable
df_horizontal[f"{plot_cols[PRED_COL]}_category"] = (df_horizontal[plot_cols[PRED_COL]]>=0.5).astype(int)
df_scope = df_horizontal.drop(plot_cols[PRED_COL], axis=1)
df_scope.head()

In [ ]:
# dropna
df_analysis = df_scope.dropna()
print(f"# samples [before,after] = {[len(df_scope),len(df_analysis)]}")

#### for category-interaction

In [ ]:
# dummies
# df_dummies = pd.get_dummies(df_analysis[dummy_cols], drop_first=True)
df_dummies = pd.get_dummies(df_analysis[dummy_cols])
df_dummied = df_analysis.merge(df_dummies, left_index=True, right_index=True)
assert len(df_analysis)==len(df_dummied), f"# samples should match before/after processing: {len(df_analysis),len(df_dummied)}"
df_dummied.head()

In [ ]:
# interaction terms
df_analysis = df_dummied.copy()
score_cols = ['MIDAS_all', 'MIDAS_text', 'MIDAS_cross', 'MIDAS_image']
# df_analysis = df_dummied.drop('MIDAS_all', axis=1)
# score_cols = ['MIDAS_text', 'MIDAS_cross', 'MIDAS_image']
category_cols = df_analysis.columns.tolist()
for col in ["id",f"{plot_cols[PRED_COL]}_category"]+score_cols+dummy_cols:
    category_cols.remove(col)
for s_col in score_cols:
    for c_col in category_cols:
        df_analysis[f"{s_col}_{c_col}"] = df_analysis[s_col]*df_analysis[c_col]
df_analysis.head()

#### shared 2

In [ ]:
# for modeling
df_analysis = df_analysis.drop(dummy_cols, axis=1)
X_cols = df_analysis.columns.tolist()
remove_cols = ["id",f"{plot_cols[PRED_COL]}_category"]
for col in remove_cols:
    X_cols.remove(col)
df_train_eval, df_test = train_test_split(df_analysis, random_state=RS, test_size=0.3)
df_train, df_eval = train_test_split(df_train_eval, random_state=RS, test_size=0.2)
print(f"[train, eval, test] size: {[df_train.shape, df_eval.shape, df_test.shape]}")
X_train_eval = df_train_eval[X_cols].values
y_train_eval = df_train_eval[f"{plot_cols[PRED_COL]}_category"].values
X_train, X_eval, X_test = df_train[X_cols].values, df_eval[X_cols].values, df_test[X_cols].values
y_train = df_train[f"{plot_cols[PRED_COL]}_category"].values
y_eval = df_eval[f"{plot_cols[PRED_COL]}_category"].values
y_test = df_test[f"{plot_cols[PRED_COL]}_category"].values

### interaction modeling

#### for all

In [ ]:
# model class
params = {
    'verbose': -1,
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'learning_rate': 0.1,
    'seed': RS,
    'deterministic':True,
    'force_row_wise':True
}

In [ ]:
# fit
lgb_train = opt_lgb.Dataset(X_train, y_train)
lgb_valid = opt_lgb.Dataset(X_eval, y_eval, reference=lgb_train)
lgb_test = opt_lgb.Dataset(X_test, y_test, reference=lgb_train)
lgb_results = {}
model = opt_lgb.LightGBMTuner(
    params=params,
    train_set=lgb_train,
    valid_sets=[lgb_train, lgb_valid],
    valid_names=['Train', 'Valid'],
    num_boost_round=500,
    early_stopping_rounds=5,
    evals_result=lgb_results,
    verbosity=-1,
    verbose_eval=-1,
    optuna_seed=RS,
)
model.run()
model = model.get_best_booster()

In [ ]:
print(model.params)

In [ ]:
# prediction
preds = model.predict(X_test)
fpr, tpr, _ = roc_curve(y_test,  preds)
auc = roc_auc_score(y_test, preds)
print("AUC="+str(auc))
plt.plot(fpr,tpr,label="AUC="+str(auc))
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.show()

In [ ]:
importance = pd.DataFrame({'feature': X_cols, 'importance': model.feature_importance()})
types = ['MIDAS_text','MIDAS_cross','MIDAS_image','model_type_O','model_type_U','model_type_V']
for t in types:
    importance[f'is_{t}']= 0
    importance.loc[importance['feature'].str.contains(t), f'is_{t}']= 1
importance = importance[importance["importance"]>=1].reset_index(drop=True)
categories = [f"is_{tp}" for tp in types]
imp_grp = importance.groupby(categories)
imp_feat = imp_grp["feature"].nunique().reset_index(drop=False)
imp_sum = imp_grp["importance"].sum().reset_index(drop=False)
print(imp_feat)
print("==================")
print(imp_sum)
imp_feat.to_csv(f"{DRIVE_DIR}/{VILIO_DIR}/{EXPORT_DIR}/all_optuna_nunique_{RS}.csv", index=False)
imp_sum.to_csv(f"{DRIVE_DIR}/{VILIO_DIR}/{EXPORT_DIR}/all_optuna_occurrences_{RS}.csv", index=False)

#### for each model type

In [ ]:
aucs = {}
df_test["preds"] = model.predict(X_test)
for tp in ["O","U","V"]:
    print(f"Model {tp}")
    if tp=="O":
        df_plot_scope = df_test[(df_test[f"model_type_U"]==0)&(df_test[f"model_type_V"]==0)].reset_index(drop=True)
    else:
        df_plot_scope = df_test[df_test[f"model_type_{tp}"]==1].reset_index(drop=True)
    fpr, tpr, _ = roc_curve(df_plot_scope[f"{plot_cols[PRED_COL]}_category"],  df_plot_scope["preds"])
    auc = roc_auc_score(df_plot_scope[f"{plot_cols[PRED_COL]}_category"],  df_plot_scope["preds"])
    print("AUC="+str(auc))
    aucs[tp] = auc
    plt.plot(fpr,tpr,label="AUC="+str(auc))
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.legend(loc=4)
    plt.show()

In [ ]:
print(aucs)

In [ ]:
aucs2 = {}
for tp in ["O","U","V"]:
    print(f"Model {tp}")
    params = {
        'verbose': -1,
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'binary_logloss',
        'learning_rate': 0.1,
        'seed': RS,
        'deterministic':True,
        'force_row_wise':True
    }
    if tp=="O":
        model_scope = (df_analysis[f"model_type_U"]==0)&(df_analysis[f"model_type_V"]==0)
    else:
        model_scope = df_analysis[f"model_type_{tp}"]==1
    df_analysis_scope = df_analysis[model_scope].reset_index(drop=True)

    df_train_eval, df_test = train_test_split(df_analysis_scope, random_state=RS, test_size=0.3)
    df_train, df_eval = train_test_split(df_train_eval, random_state=RS, test_size=0.2)
    print(f"[train, eval, test] size: {[df_train.shape, df_eval.shape, df_test.shape]}")
    X_train_eval = df_train_eval[X_cols].values
    y_train_eval = df_train_eval[f"{plot_cols[PRED_COL]}_category"].values
    X_train, X_eval, X_test = df_train[X_cols].values, df_eval[X_cols].values, df_test[X_cols].values
    y_train = df_train[f"{plot_cols[PRED_COL]}_category"].values
    y_eval = df_eval[f"{plot_cols[PRED_COL]}_category"].values
    y_test = df_test[f"{plot_cols[PRED_COL]}_category"].values

    lgb_train = opt_lgb.Dataset(X_train, y_train)
    lgb_valid = opt_lgb.Dataset(X_eval, y_eval, reference=lgb_train)
    lgb_test = opt_lgb.Dataset(X_test, y_test, reference=lgb_train)

    lgb_results = {}
    model = opt_lgb.LightGBMTuner(
        params=params,
        train_set=lgb_train,
        valid_sets=[lgb_train, lgb_valid],
        valid_names=['Train', 'Valid'],
        num_boost_round=500,
        early_stopping_rounds=5,
        evals_result=lgb_results,
        verbosity=-1,
        verbose_eval=-1,
        optuna_seed=RS,
    )
    model.run()
    model = model.get_best_booster()
    print(model.params)
    print("------------------")

    preds = model.predict(X_test)
    fpr, tpr, _ = roc_curve(y_test,  preds)
    auc = roc_auc_score(y_test, preds)
    print("AUC="+str(auc))
    aucs2[tp] = auc
    plt.plot(fpr,tpr,label="AUC="+str(auc))
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.legend(loc=4)
    plt.show()
    print("------------------")

    importance = pd.DataFrame({'feature': X_cols, 'importance': model.feature_importance()})
    submodels = df_vilio["submodel"].unique().tolist()
    submodel_cols = [f"submodel_{model}" for model in submodels]
    types = ['MIDAS_text','MIDAS_cross','MIDAS_image']+submodel_cols
    for t in types:
        importance[f'is_{t}']= 0
        importance.loc[importance['feature'].str.contains(t), f'is_{t}']= 1
    importance = importance[importance["importance"]>=1].reset_index(drop=True)
    categories = [f"is_{tp}" for tp in types]
    imp_grp = importance.groupby(categories)
    imp_feat = imp_grp["feature"].nunique().reset_index(drop=False)
    imp_sum = imp_grp["importance"].sum().reset_index(drop=False)
    print(imp_feat)
    print("------------------")
    print(imp_sum)
    print("==================")
    imp_feat.to_csv(f"{DRIVE_DIR}/{VILIO_DIR}/{EXPORT_DIR}/{tp}_indv_optuna_nunique_{RS}.csv", index=False)
    imp_sum.to_csv(f"{DRIVE_DIR}/{VILIO_DIR}/{EXPORT_DIR}/{tp}_indv_optuna_occurrences_{RS}.csv", index=False)

In [ ]:
print(aucs2)

## export

In [ ]:
dfs = lgb_results2df()
print(dfs.keys())
for ky in dfs.keys():
    print(ky)
    print(dfs[ky].shape)

In [ ]:
key_cols = dfs["_indv_optuna_occurrences_"].drop("importance", axis=1).columns.tolist()
df_merged = dfs["_indv_optuna_nunique_"].merge(dfs["_indv_optuna_occurrences_"].set_index(key_cols), left_on=key_cols, right_index=True)
df_merged.head()

In [ ]:
df_merged.to_csv(f"{DRIVE_DIR}/{VILIO_DIR}/{EXPORT_DIR}/indv_optuna_merged.csv", index=False)